In [1]:
import pandas as pd
from functools import reduce
import itertools
from itertools import chain
import pickle
import torch
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
from sklearn.metrics.cluster import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import numpy as np
import regex as re
import igraph
import sys
from pytictoc import TicToc
import statistics
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from pandas import DataFrame

In [37]:
time = TicToc()

In [38]:
DAG = pd.read_csv("data/ruleBased_DAGs.csv",sep =',')
features = pd.read_csv("data/aggFeatures.csv",sep =',')
y_vector_df = pd.read_csv("data/vectorDF.csv")

In [39]:
DAGs['cat_run_time'] = pd.cut(DAGs.run_time, bins = list(range(0,3600,100)), labels=np.arange(35), right=False)

In [40]:
embedDf = pd.read_csv("data/embedding_model_multisimilarityLoss_11Ys_30Epochs_train_test_stratified.csv")
embedDf180 = pd.read_csv("data/model_MultiSimilarityLoss_11Ys_70Epochs_train_test_stratified_3Gpoolinglayers180.csv")
embedDf64 = pd.read_csv("data/model_C_TripletMarginLoss2_distanceCosineSimilarity_11Ys_70Epochs_train_test_stratified_globalavg64.csv")
embedDf128 = pd.read_csv("data/model_C_TripletMarginLoss2_distanceCosineSimilarity_11Ys_70Epochs_train_test_stratified_globalavg128.csv")
embedDf16 = pd.read_csv("data/model_C_TripletMarginLoss2_distanceCosineSimilarity_11Ys_70Epochs_train_test_stratified_globalavg16.csv")

In [41]:
embedDf = embedDf.drop(['Unnamed: 0'], axis=1)
embedDf.fillna(method ='ffill', inplace = True) 

In [42]:
### breaking the embedding dataframe in three fields: max-avg-add for different embedding size
embedDf_max = embedDf.iloc[:,0:32]
embedDf_avg = embedDf.iloc[:,32:64]
embedDf_add = embedDf.iloc[:,64:96]
embedDf_all = embedDf.iloc[:,0:96]
embedDf_max_60 = embedDf180.iloc[:,1:60]
embedDf_avg_60 = embedDf180.iloc[:,60:120]
embedDf_add_60 = embedDf180.iloc[:,120:180]
embedDf_all_60 = embedDf180.iloc[:,1:180]
embedDf_avg64 = embedDf64.iloc[:,0:64]
embedDf_avg128 = embedDf128.iloc[:,1:129]
embedDf_avg16 = embedDf16.iloc[:,1:17]

In [43]:
df = embedDf.iloc[:,0:96]

In [44]:
### Standardization and Normalization ###

scaler = StandardScaler() 


#size of 32
DF_scaled_max = scaler.fit_transform(embedDf_max)
DF_scaled_avg = scaler.fit_transform(embedDf_avg)
DF_scaled_add = scaler.fit_transform(embedDf_add)
DF_scaled_all = scaler.fit_transform(embedDf_all)


#size of 60
DF_scaled_max_60 = scaler.fit_transform(embedDf_max_60)
DF_scaled_avg_60 = scaler.fit_transform(embedDf_avg_60)
DF_scaled_add_60 = scaler.fit_transform(embedDf_add_60)



#different sized of AVG
DF_scaled_avg16 = scaler.fit_transform(embedDf_avg16)
DF_scaled_avg64 = scaler.fit_transform(embedDf_avg64)
DF_scaled_avg128 = scaler.fit_transform(embedDf_avg128)

In [45]:
# Normalization

#size of 32
DF_normalized_max = normalize(DF_scaled_max)
DF_normalized_avg = normalize(DF_scaled_avg) 
DF_normalized_add = normalize(DF_scaled_add) 
DF_normalized_all = normalize(DF_scaled_all) 

#size of 60
DF_normalized_max_60 = normalize(DF_scaled_max_60)
DF_normalized_avg_60 = normalize(DF_scaled_avg_60) 
DF_normalized_add_60 = normalize(DF_scaled_add_60) 


#different sized of AVG
DF_normalized_avg16 = normalize(DF_scaled_avg16)
DF_normalized_avg64 = normalize(DF_scaled_avg64) 
DF_normalized_avg128 = normalize(DF_scaled_avg128) 

In [46]:
#time.tic()
#optics_model = OPTICS()
#optics_avg_64 = optics_model.fit(DF_normalized_avg64)
#pickle.dump(optics_avg_64, open('models/optics_avg64_clusterSize_2_only20nodes.sav', 'wb'))
#time.toc()

In [47]:
#### First Checkpoint ####
#The model is fitted with all the eleven samples described above# 

In [48]:
optic_max = pickle.load(open('models/optic_max.sav', 'rb'))
optic_add = pickle.load(open('models/optic_add.sav', 'rb'))
optic_all = pickle.load(open('models/optic_all.sav', 'rb'))
optic_avg = pickle.load(open('models/optic_avg.sav', 'rb'))
optic_avg16 = pickle.load(open('models/optic_avg16.sav', 'rb'))
optic_avg64 = pickle.load(open('models/optic__avg_64.sav', 'rb'))
optic_avg128 = pickle.load(open('models/optic__avg_128.sav', 'rb'))
optic_max_60 = pickle.load(open('models/optic_max_60.sav', 'rb'))
optic_add_60 = pickle.load(open('models/optic_add_60.sav', 'rb'))
optic_avg_60 = pickle.load(open('models/optic_avg_60.sav', 'rb'))

/home/alireza.alamgiralem/.local/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator OPTICS from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [49]:
#32
labels_max = list(optic_max.labels_)
labels_avg = list(optic_avg.labels_)
labels_add = list(optic_add.labels_)
labels_all = list(optic_all.labels_)

#60
labels_max_60 = list(optic_max_60.labels_)
labels_avg_60 = list(optic_avg_60.labels_)
labels_add_60 = list(optic_add_60.labels_)


#only avg
labels_avg16 = list(optic_avg16.labels_)
labels_avg64 = list(optic_avg64.labels_)
lables_avg128 = list(optic_avg128.labels_)

In [50]:
#assigning different labels to a single the dataframe

#32
embedDf = embedDf.assign(OPTIC_labels_max = labels_max)
embedDf = embedDf.assign(OPTIC_labels_avg = labels_avg)
embedDf = embedDf.assign(OPTIC_labels_add = labels_add)
embedDf = embedDf.assign(OPTIC_labels_all = labels_all)

#60
embedDf = embedDf.assign(OPTIC_labels_max_60 = labels_max_60)
embedDf = embedDf.assign(OPTIC_labels_avg_60 = labels_avg_60)
embedDf = embedDf.assign(OPTIC_labels_add_60 = labels_add_60)

#avg
embedDf = embedDf.assign(OPTIC_labels_avg16 = labels_avg16)
embedDf = embedDf.assign(OPTIC_labels_avg64 = labels_avg64)
embedDf = embedDf.assign(OPTIC_labels_avg128 = lables_avg128)

In [51]:
DAGs = pd.merge(DAG, features, how='inner', on = 'job_name')
DAGs = DAGs.drop('Unnamed: 0', axis=1)
joinTable = pd.merge(DAGs, embedDf, on = 'job_name')

In [52]:
#calculation of group sizes for different clusters
joinTable['lableSize_group'] = joinTable.groupby('group')['group'].transform('size')

joinTable['lableSize_max'] = joinTable.groupby('OPTIC_labels_max')['OPTIC_labels_max'].transform('size')
joinTable['lableSize_avg'] = joinTable.groupby('OPTIC_labels_avg')['OPTIC_labels_avg'].transform('size')
joinTable['lableSize_add'] = joinTable.groupby('OPTIC_labels_add')['OPTIC_labels_add'].transform('size')
joinTable['lableSize_all'] = joinTable.groupby('OPTIC_labels_all')['OPTIC_labels_all'].transform('size')


joinTable['lableSize_max_60'] = joinTable.groupby('OPTIC_labels_max_60')['OPTIC_labels_max_60'].transform('size')
joinTable['lableSize_avg_60'] = joinTable.groupby('OPTIC_labels_avg_60')['OPTIC_labels_avg_60'].transform('size')
joinTable['lableSize_add_60'] = joinTable.groupby('OPTIC_labels_add_60')['OPTIC_labels_add_60'].transform('size')


joinTable['lableSize_avg16'] = joinTable.groupby('OPTIC_labels_avg16')['OPTIC_labels_avg16'].transform('size')
joinTable['lableSize_avg64'] = joinTable.groupby('OPTIC_labels_avg64')['OPTIC_labels_avg64'].transform('size')
joinTable['lableSize_avg128'] = joinTable.groupby('OPTIC_labels_avg128')['OPTIC_labels_avg128'].transform('size')

In [53]:
set_group = list(joinTable.group.unique())

#32
set_lables_max = list(joinTable.OPTIC_labels_max.unique())
set_lables_avg = list(joinTable.OPTIC_labels_avg.unique())
set_lables_add = list(joinTable.OPTIC_labels_add.unique())
set_lables_all = list(joinTable.OPTIC_labels_all.unique())

#60
set_lables_max_60 = list(joinTable.OPTIC_labels_max_60.unique())
set_lables_avg_60 = list(joinTable.OPTIC_labels_avg_60.unique())
set_lables_add_60 = list(joinTable.OPTIC_labels_add_60.unique())

set_lables_avg16 = list(joinTable.OPTIC_labels_avg16.unique())
set_lables_avg64 = list(joinTable.OPTIC_labels_avg64.unique())
set_lables_avg128 = list(joinTable.OPTIC_labels_avg128.unique())


In [60]:
## Calculation of variance with outliers

#calculating the variance for every group (by OPTIC_max)
var_by_optic_max = list()
for i in set_lables_max:
    df = joinTable[joinTable.OPTIC_labels_max == i]
    var_by_optic_max.append(np.var(df.run_time))   
print('var by max = ',np.mean(var_by_optic_max))

#calculating the variance for every group (by OPTIC_avg)
var_by_optic_avg = list()
for i in set_lables_avg:
    df = joinTable[joinTable.OPTIC_labels_avg == i]
    var_by_optic_avg.append(np.var(df.run_time))   
print('var by avg = ',np.mean(var_by_optic_avg))

#calculating the variance for every group (by OPTIC_add)
var_by_optic_add = list()
for i in set_lables_add:
    df = joinTable[joinTable.OPTIC_labels_add == i]
    var_by_optic_add.append(np.var(df.run_time))   
print('var by add = ',np.mean(var_by_optic_add))

#calculating the variance for every group (by OPTIC_add)
var_by_optic_all = list()
for i in set_lables_all:
    df = joinTable[joinTable.OPTIC_labels_all == i]
    var_by_optic_all.append(np.var(df.run_time))   
print('var by all = ',np.mean(var_by_optic_all))

print('\n\n embedding size of 60')
#########60
#calculating the variance for every group (by OPTIC_max)
var_by_optic_max_60 = list()
for i in set_lables_max_60:
    df = joinTable[joinTable.OPTIC_labels_max_60 == i]
    var_by_optic_max_60.append(np.var(df.run_time))   
print('var by max_60 = ',np.mean(var_by_optic_max_60))

#calculating the variance for every group (by OPTIC_avg)
var_by_optic_avg_60 = list()
for i in set_lables_avg_60:
    df = joinTable[joinTable.OPTIC_labels_avg_60 == i]
    var_by_optic_avg_60.append(np.var(df.run_time))   
print('var by avg_60 = ',np.mean(var_by_optic_avg_60))

#calculating the variance for every group (by OPTIC_add)
var_by_optic_add_60 = list()
for i in set_lables_add_60:
    df = joinTable[joinTable.OPTIC_labels_add_60 == i]
    var_by_optic_add_60.append(np.var(df.run_time))   
print('var by add_60 = ',np.mean(var_by_optic_add_60))


print('\n\nlabels generated by other settings')



#calculating the variance for every group (by OPTIC_avg16)
var_by_optic_avg16 = list()
for i in set_lables_avg16:
    df = joinTable[joinTable.OPTIC_labels_avg16 == i]
    var_by_optic_avg16.append(np.var(df.run_time))   
print('var by avg 16 = ',np.mean(var_by_optic_avg16))

#calculating the variance for every group (by OPTIC_avg64)
var_by_optic_avg64 = list()
for i in set_lables_avg64:
    df = joinTable[joinTable.OPTIC_labels_avg64 == i]
    var_by_optic_avg64.append(np.var(df.run_time))   
print('var by avg 64 = ',np.mean(var_by_optic_avg64))

#calculating the variance for every group (by OPTIC_avg128)
var_by_optic_avg128 = list()
for i in set_lables_avg128:
    df = joinTable[joinTable.OPTIC_labels_avg128 == i]
    var_by_optic_avg128.append(np.var(df.run_time))   
print('var by avg 128 = ',np.mean(var_by_optic_avg128))


var by max =  49953.59310954847
var by avg =  46965.61294763679
var by add =  47012.049164246746
var by all =  46691.360406226566


 embedding size of 60
var by max_60 =  43020.78508412694
var by avg_60 =  37046.70788939994
var by add_60 =  36408.29908553091


labels generated by other settings
var by avg 16 =  30622.31521614127
var by avg 64 =  30465.134276891633
var by avg 128 =  39938.6161891401


In [57]:
def outlierdeletor (cluster_list):
    if -1 in cluster_list:
        cluster_list.remove(-1)
    return cluster_list
#32
set_lables_max = outlierdeletor(set_lables_max)
set_lables_avg = outlierdeletor(set_lables_avg)
set_lables_add = outlierdeletor(set_lables_add)
set_lables_all = outlierdeletor(set_lables_all)

#60
set_lables_max_60 = outlierdeletor(set_lables_max_60)
set_lables_avg_60 = outlierdeletor(set_lables_avg_60)
set_lables_add_60 = outlierdeletor(set_lables_add_60)

#avg
set_lables_avg16 = outlierdeletor(set_lables_avg16)
set_lables_avg64 = outlierdeletor(set_lables_avg64)
set_lables_avg128 = outlierdeletor(set_lables_avg128)

In [58]:
# Rule-based
var_by_group_inlier = list()
for i in set_group:
    df = joinTable[(joinTable.group == i) & (joinTable.lableSize_group >= 2) ]
    var_by_group_inlier.append(np.var(df.run_time))
print('var by previous groups = ',np.nanmean(var_by_group_inlier))

print('beseline (group size bigger than 5)')
var_by_group_inlier_5 = list()
for i in set_group:
    df = joinTable[(joinTable.group == i) & (joinTable.lableSize_group >= 5) ]
    var_by_group_inlier_5.append(np.var(df.run_time))
print('var by previous groups = ',np.nanmean(var_by_group_inlier_5))

print('\n\nembeddings size of 32')

#######32######
#calculating the variance for every group (by OPTIC_max)
var_by_optic_max_inlier = list()
for i in set_lables_max:
    df = joinTable[joinTable.OPTIC_labels_max == i]
    var_by_optic_max_inlier.append(np.var(df.run_time))   
print('var by max = ',np.mean(var_by_optic_max_inlier))

#calculating the variance for every group (by OPTIC_avg)
var_by_optic_avg_inlier = list()
for i in set_lables_avg:
    df = joinTable[joinTable.OPTIC_labels_avg == i]
    var_by_optic_avg_inlier.append(np.var(df.run_time))   
print('var by avg = ',np.mean(var_by_optic_avg_inlier))

#calculating the variance for every group (by OPTIC_add)
var_by_optic_add_inlier = list()
for i in set_lables_add:
    df = joinTable[joinTable.OPTIC_labels_add == i]
    var_by_optic_add_inlier.append(np.var(df.run_time))   
print('var by add = ',np.mean(var_by_optic_add_inlier))

#calculating the variance for every group (by OPTIC_add)
var_by_optic_all_inlier = list()
for i in set_lables_all:
    df = joinTable[joinTable.OPTIC_labels_all == i]
    var_by_optic_all_inlier.append(np.var(df.run_time))   
print('var by all = ',np.mean(var_by_optic_all_inlier))

print('\n\n embedding size of 60')
#########60
#calculating the variance for every group (by OPTIC_max)
var_by_optic_max_60_inlier = list()
for i in set_lables_max_60:
    df = joinTable[joinTable.OPTIC_labels_max_60 == i]
    var_by_optic_max_60_inlier.append(np.var(df.run_time))   
print('var by max_60 = ',np.mean(var_by_optic_max_60_inlier))

#calculating the variance for every group (by OPTIC_avg)
var_by_optic_avg_60_inlier = list()
for i in set_lables_avg_60:
    df = joinTable[joinTable.OPTIC_labels_avg_60 == i]
    var_by_optic_avg_60_inlier.append(np.var(df.run_time))   
print('var by avg_60 = ',np.mean(var_by_optic_avg_60_inlier))

#calculating the variance for every group (by OPTIC_add)
var_by_optic_add_60_inlier = list()
for i in set_lables_add_60:
    df = joinTable[joinTable.OPTIC_labels_add_60 == i]
    var_by_optic_add_60_inlier.append(np.var(df.run_time))   
print('var by add_60 = ',np.mean(var_by_optic_add_60_inlier))


print('\n\n different sizes of average')
#calculating the variance for every group (by OPTIC_avg16)
var_by_optic_avg16_inlier = list()
for i in set_lables_avg16:
    df = joinTable[joinTable.OPTIC_labels_avg16 == i]
    var_by_optic_avg16_inlier.append(np.var(df.run_time))   
print('var by avg 16 = ',np.mean(var_by_optic_avg16_inlier))

#calculating the variance for every group (by OPTIC_avg64)
var_by_optic_avg64_inlier = list()
for i in set_lables_avg64:
    df = joinTable[joinTable.OPTIC_labels_avg64 == i]
    var_by_optic_avg64_inlier.append(np.var(df.run_time))   
print('var by avg 64 = ',np.mean(var_by_optic_avg64_inlier))

#calculating the variance for every group (by OPTIC_avg128)
var_by_optic_avg128_inlier = list()
for i in set_lables_avg128:
    df = joinTable[joinTable.OPTIC_labels_avg128 == i]
    var_by_optic_avg128_inlier.append(np.var(df.run_time))   
print('var by avg 128 = ',np.mean(var_by_optic_avg128_inlier))


var by previous groups =  40823.22516285953
beseline (group size bigger than 5)
var by previous groups =  50127.79864920701


embeddings size of 32
var by max =  49953.59310954847
var by avg =  46965.61294763679
var by add =  47012.049164246746
var by all =  46691.360406226566


 embedding size of 60
var by max_60 =  43020.78508412694
var by avg_60 =  37046.70788939994
var by add_60 =  36408.29908553091


 different sizes of average
var by avg 16 =  30622.31521614127
var by avg 64 =  30465.134276891633
var by avg 128 =  39938.6161891401


In [61]:
#exploring lables

print ('description of baseline groups smallest group size of 2 \nnumber of groups =', len(set_group))
df = joinTable[joinTable.lableSize_group >= 2]
df = df.drop_duplicates(subset=['group'])
print('biggest group size = ',max(df.lableSize_group), '\navg size of groups = ', round(np.mean(df.lableSize_group)))
print('number of outliers = ' ,len(joinTable[joinTable.lableSize_group < 2]))
print('runtime variation of outliers = ', np.var(joinTable[joinTable.lableSize_group < 2].run_time)) 
print('runtime variation of inliers = ', np.nanmean(var_by_group_inlier),'\n\n') 

print ('description of baseline groups smallest group size of 5 \nnumber of groups =', len(set_group))
df = joinTable[joinTable.lableSize_group >= 5]
df = df.drop_duplicates(subset=['group'])
print('biggest group size = ',max(df.lableSize_group),'\navg size of groups = ', round(np.mean(df.lableSize_group)))
print('number of outliers = ' ,len(joinTable[joinTable.lableSize_group < 5]))
print('runtime variation of outliers = ', np.var(joinTable[joinTable.lableSize_group < 5].run_time)) 
print('runtime variation of inliers = ', np.nanmean(var_by_group_inlier_5),'\n\n')

print('description of labeles with embedding size of 32 (MAX)\nnumber of groups = ', len(set_lables_max))
df = joinTable[joinTable.OPTIC_labels_max != -1]
df = df.drop_duplicates(subset=['OPTIC_labels_max'])
print('biggest group size = ',max(df.lableSize_max), '\navg size of groups = ', round(np.mean(df.lableSize_max)))
print('number of outliers = ' ,len(joinTable[joinTable.OPTIC_labels_max == -1]))
print('mean runtime variation of outliers = ', np.mean(var_by_optic_max)) 
print('mean runtime variation of inliers = ', np.mean(var_by_optic_max_inlier),'\n\n') 

print('description of labeles with embedding size of 32 (AVG)\nnumber of groups = ', len(set_lables_max))
df = joinTable[joinTable.OPTIC_labels_avg != -1]
df = df.drop_duplicates(subset=['OPTIC_labels_avg'])
print('biggest group size = ',max(df.lableSize_avg), '\navg size of groups = ', round(np.mean(df.lableSize_avg)))
print('number of outliers = ' ,len(joinTable[joinTable.OPTIC_labels_avg == -1]))
print('mean runtime variation of outliers = ', np.mean(var_by_optic_avg)) 
print('mean runtime variation of inliers = ', np.mean(var_by_optic_avg_inlier),'\n\n') 

print('description of labeles with embedding size of 32 (ADD)\nnumber of groups = ', len(set_lables_max))
df = joinTable[joinTable.OPTIC_labels_add != -1]
df = df.drop_duplicates(subset=['OPTIC_labels_add'])
print('biggest group size = ',max(df.lableSize_add), '\navg size of groups = ', round(np.mean(df.lableSize_add)))
print('number of outliers = ' ,len(joinTable[joinTable.OPTIC_labels_add == -1]))
print('mean runtime variation of outliers = ', np.mean(var_by_optic_add)) 
print('mean runtime variation of inliers = ', np.mean(var_by_optic_add_inlier),'\n\n')

print('description of labeles with embedding size of 32 (all) (96)\nnumber of groups = ', len(set_lables_all))
df = joinTable[joinTable.OPTIC_labels_all != -1]
df = df.drop_duplicates(subset=['OPTIC_labels_all'])
print('biggest group size = ',max(df.lableSize_all), '\navg size of groups = ', round(np.mean(df.lableSize_all)))
print('number of outliers = ' ,len(joinTable[joinTable.OPTIC_labels_all == -1]))
print('mean runtime variation of outliers = ', np.mean(var_by_optic_all)) 
print('mean runtime variation of inliers = ', np.mean(var_by_optic_all_inlier),'\n\n')

print('description of labeles with embedding size of 60 (MAX) \nnumber of groups = ', len(set_lables_max_60))
df = joinTable[joinTable.OPTIC_labels_max_60 != -1]
df = df.drop_duplicates(subset=['OPTIC_labels_max_60'])
print('biggest group size = ',max(df.lableSize_max_60), '\navg size of groups = ', round(np.mean(df.lableSize_max_60)))
print('number of outliers = ' ,len(joinTable[joinTable.OPTIC_labels_max_60 == -1]))
print('mean runtime variation of outliers = ', np.mean(var_by_optic_max_60)) 
print('mean runtime variation of inliers = ', np.mean(var_by_optic_max_60_inlier),'\n\n')

print('description of labeles with embedding size of 60 (AVG) \nnumber of groups = ', len(set_lables_avg_60))
df = joinTable[joinTable.OPTIC_labels_avg_60 != -1]
df = df.drop_duplicates(subset=['OPTIC_labels_avg_60'])
print('biggest group size = ',max(df.lableSize_avg_60), '\navg size of groups = ', round(np.mean(df.lableSize_avg_60)))
print('number of outliers = ' ,len(joinTable[joinTable.OPTIC_labels_avg_60 == -1]))
print('mean runtime variation of outliers = ', np.mean(var_by_optic_avg_60)) 
print('mean runtime variation of inliers = ', np.mean(var_by_optic_avg_60_inlier),'\n\n')

print('description of labeles with embedding size of 60 (ADD) \nnumber of groups = ', len(set_lables_add_60))
df = joinTable[joinTable.OPTIC_labels_add_60 != -1]
df = df.drop_duplicates(subset=['OPTIC_labels_add_60'])
print('biggest group size = ',max(df.lableSize_add_60), '\navg size of groups = ', round(np.mean(df.lableSize_add_60)))
print('number of outliers = ' ,len(joinTable[joinTable.OPTIC_labels_add_60 == -1]))
print('mean runtime variation of outliers = ', np.mean(var_by_optic_add_60)) 
print('mean runtime variation of inliers = ', np.mean(var_by_optic_add_60_inlier),'\n\n')

print('description of labeles with embedding size of 16 (AVG) \nnumber of groups = ', len(set_lables_avg16))
df = joinTable[joinTable.OPTIC_labels_avg16 != -1]
df = df.drop_duplicates(subset=['OPTIC_labels_avg16'])
print('biggest group size = ',max(df.lableSize_avg16), '\navg size of groups = ', round(np.mean(df.lableSize_avg16)))
print('number of outliers = ' ,len(joinTable[joinTable.OPTIC_labels_avg16 == -1]))
print('mean runtime variation of outliers = ', np.mean(var_by_optic_avg16)) 
print('mean runtime variation of inliers = ', np.mean(var_by_optic_avg16_inlier),'\n\n')

print('description of labeles with embedding size of 64 (AVG) \nnumber of groups = ', len(set_lables_avg64))
df = joinTable[joinTable.OPTIC_labels_avg64 != -1]
df = df.drop_duplicates(subset=['OPTIC_labels_avg64'])
print('biggest group size = ',max(df.lableSize_avg64), '\navg size of groups = ', round(np.mean(df.lableSize_avg64)))
print('number of outliers = ' ,len(joinTable[joinTable.OPTIC_labels_avg64 == -1]))
print('mean runtime variation of outliers = ', np.mean(var_by_optic_avg64)) 
print('mean runtime variation of inliers = ', np.mean(var_by_optic_avg64_inlier),'\n\n')

print('description of labeles with embedding size of 128 (AVG) \nnumber of groups = ', len(set_lables_avg128))
df = joinTable[joinTable.OPTIC_labels_avg128 != -1]
df = df.drop_duplicates(subset=['OPTIC_labels_avg128'])
print('biggest group size = ',max(df.lableSize_avg128), '\navg size of groups = ', round(np.mean(df.lableSize_avg128)))
print('number of outliers = ' ,len(joinTable[joinTable.OPTIC_labels_avg128 == -1]))
print('mean runtime variation of outliers = ', np.mean(var_by_optic_avg128)) 
print('mean runtime variation of inliers = ', np.mean(var_by_optic_avg128_inlier),'\n\n')



description of baseline groups smallest group size of 2 
number of groups = 24961
biggest group size =  1904 
avg size of groups =  9
number of outliers =  17117
runtime variation of outliers =  283197.05708497413
runtime variation of inliers =  40823.22516285953 


description of baseline groups smallest group size of 5 
number of groups = 24961
biggest group size =  1904 
avg size of groups =  17
number of outliers =  56759
runtime variation of outliers =  264161.92108860996
runtime variation of inliers =  50127.79864920701 


description of labeles with embedding size of 32 (MAX)
number of groups =  13619
biggest group size =  2488 
avg size of groups =  16
number of outliers =  9279
mean runtime variation of outliers =  49953.59310954847
mean runtime variation of inliers =  49953.59310954847 


description of labeles with embedding size of 32 (AVG)
number of groups =  13619
biggest group size =  2016 
avg size of groups =  15
number of outliers =  10320
mean runtime variation of ou